##  Sentinel-1  geometric and radiometric terrain correction

In this tutorial, you will learn how to generate a Sentinel-1 geometric terrain correct product (GTC) and radiometric terrain correct product (RTC) from a GRD Sentinel-1 image using [sarsen](https://github.com/bopen/sarsen).

We will:
- Download an example Sentinel-1 GRD on South-of-Redmond (Seattle, US)
- Download the DEM on South-of-Redmond (Seattle, US) area
- Compute the GTC with [sarsen](https://github.com/bopen/sarsen)
- Compute the RTC with [sarsen](https://github.com/bopen/sarsen)
- Comparate the GTC to the RTC
- Comparate the RTC computed with [sarsen](https://github.com/bopen/sarsen) to the RTC already available in the Planatery Computer

### Introduction
 The typical side-looking SAR system acquires with uniform sampling in azimuth and in slant range, where the azimuth represents the time when a given target is acquired and the range represents the absolute sensor-to-target distance.
Because of this, the near range appears compressed with respect to the far range. Furthermore, any deviation of the target elevation from a smooth geoid results in additional local geometric and radiometric distortions known as foreshortening and layover and shadow. Radar foreshortening is the effect of imaged terrain surfaces sloping towards the radar appearing shortened relative to those sloping away from the radar. Radar layover is an extreme case of foreshortening that occurs when the slope of the terrain is greater than the angle of the incident signal. 


### Geocoded Terrain Corrected (GTC) product
The GRD Sentinel-1 product already provides a geometric correction that removes the compression effect on the near range, converting the data from the slant-range to ground-range coordinates with uniform sampling in ground-range. 

The GTC product also provides the correction of the distortions due to target elevation. While the GRD can be computed without using a digital elevation model (DEM), the GTC processing requires a DEM to associate the ground points to the image points. 


### Radiometric Terrain Corrected (RTC) product
Terrain variations do not affect the position of a given point on the Earth's surface only, but they also affect the brightness of the radar return. The [sarsen](https://pypi.org/project/sarsen/) radiometric terrain correction compensates for the backscatter modulation generated by the topography of the scene. This produces a more uniform backscatter image, emphasizing the radiometric differences of the terrain.

<hr style="border:2px solid blue"> </hr>

Dependecies: sarsen 

In [ ]:
# pip install sarsen

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams["font.size"] = 12

In [ ]:
import os
import tempfile

# enable the `.rio` accessor
import rioxarray  # noqa: F401
import xarray as xr

from sarsen import apps, scene

### Processing definition

In [ ]:
# create a temporary directory where to store downloaded data
tmp_dir = tempfile.gettempdir()

# DEM paths
dem_urlpath = os.path.join(tmp_dir, "South-of-Redmond-10m.tif")
dem_UTM_urlpath = os.path.join(tmp_dir, "South-of-Redmond-10m_UTM.tif")

# path to Sentinel-1 input product in the Planetary Computer
product_folder = "GRD/2021/12/17/IW/DV/S1B_IW_GRDH_1SDV_20211217T141304_20211217T141329_030066_039705_9048"  # noqa: E501

# band to be processed
measurement_group = "IW/VV"

### Download DEM of South-of-Redmond (Seattle, US)

In [ ]:
import adlfs
import planetary_computer
import pystac_client
import stackstac

#### Area of interest definition

In [ ]:
seattle_lon, seattle_lat = [-121.95, 47.04]
buffer = 0.2
bbox = [
    seattle_lon - buffer,
    seattle_lat - buffer,
    seattle_lon + buffer,
    seattle_lat + buffer,
]

#### Download DEMs

Multiple DEMs acquired at different times are available on this area

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)
threedep = catalog.get_child(id="3dep-seamless")

search = catalog.search(collections="3dep-seamless", bbox=bbox)
items = list(search.get_items())

In [ ]:
items_high_res = [
    planetary_computer.sign(item).to_dict()
    for item in items
    if item.properties["gsd"] == 10
]

dem_raster_all = stackstac.stack(items_high_res, bounds=bbox).squeeze()
dem_raster_all

#### Average the DEMs along time

In [ ]:
dem_raster = dem_raster_all.compute()
if "time" in dem_raster.dims:
    dem_raster = dem_raster.mean("time")
dem_raster.rio.set_crs(dem_raster_all.rio.crs)
dem_raster.rio.to_raster(dem_urlpath)

#### Convert the DEM in UTM coordinates

The DEM conversion in UTM is done to simplify the comparison between [sarsen](https://pypi.org/project/sarsen/) RTC with the RTC provided on the Planatery Computer.

In [ ]:
dem_raster = scene.open_dem_raster(dem_urlpath)

# find the UTM zone and project in UTM
t_srs = dem_raster.rio.estimate_utm_crs()
dem_raster_10m_UTM = dem_raster.rio.reproject(t_srs, resolution=(10, 10))

# crop DEM to our area of interest and save it
dem_UTM_corners = dict(x=slice(565000, 594000), y=slice(5220000, 5190000))

dem_raster_10m_UTM = dem_raster_10m_UTM.sel(**dem_UTM_corners)
dem_raster_10m_UTM.rio.to_raster(dem_UTM_urlpath)

In [ ]:
dem_raster_10m_UTM.plot()
plt.title("DEM in UTM coordinates")

### Download Sentinel-1 GRD

Find the target image in the Planetary Computer

- Connect to the Planetary Computer
- List the content of the Sentinel-1 folder product 
- Download the product

In [ ]:
grd_account_name = "sentinel1euwest"
grd_storage_container = "s1-grd"
grd_token = planetary_computer.sas.get_token(
    grd_account_name, grd_storage_container
).token

grd_product_folder = f"{grd_storage_container}/{product_folder}"

grd_fs = adlfs.AzureBlobFileSystem(grd_account_name, credential=grd_token)
grd_fs.ls(grd_product_folder)

In [ ]:
grd_local_path = os.path.join(tmp_dir, product_folder)
grd_fs.get(f"{grd_storage_container}/{product_folder}", grd_local_path, recursive=True)

### Processing

#### GTC

Here we compute the geometric terrain correction.

The processing steps can be summarized as follows: 
- Pre-process DEM: convert DEM coordinates to [Earth Centred Earth Fixed (ECEF)](https://en.wikipedia.org/wiki/Earth-centered,_Earth-fixed_coordinate_system) reference system
- Simulate the acquisition: compute the image coordinates of each DEM point by resolving the zero-doppler equations 
- Calibrate the radiometry: compute the beta nought intensity 
- Interpolate image: interpolate the SAR image on the DEM coordinates.

The output is the input SAR image resampled on DEM coordinates. 

Input parameters:
- `product_urlpath`: product path
- `measurement_group`: band to be processed in the form {swath}/{polarization} (for more details see [xarray-sentinel](https://pypi.org/project/xarray-sentinel/))
- `dem_urlpath`: path to the input DEM. [sarsen](https://pypi.org/project/sarsen/) supports all the DEMs supported by GDAL/Proj for ECEF-translation. 
- `interp_method`: interpolation method,  [sarsen](https://pypi.org/project/sarsen/) supports all the interpolation methods supported by [xarray.Dataset.interp](https://xarray.pydata.org/en/stable/generated/xarray.Dataset.interp.html)
- `chunks`: dask chunks
- `output_urlpath`: output path

In [ ]:
gtc_path = os.path.basename(product_folder) + ".10m.GTC.tif"

apps.backward_geocode_sentinel1(
    product_urlpath=grd_local_path,
    measurement_group=measurement_group,
    dem_urlpath=dem_UTM_urlpath,
    interp_method="nearest",
    chunks={"slant_range_time": 2048},
    output_urlpath=gtc_path,
)

In [ ]:
gtc = xr.open_dataarray(gtc_path).drop("band")
_ = gtc.plot(vmax=0.4)

#### RTC
[sarsen](https://pypi.org/project/sarsen/) implements the [Gamma Flattening](https://ieeexplore.ieee.org/document/5752845) for the radiometric terrain correction algorithm.

Each pixel of the SAR image is normalised with respect to its corresponding area on the plane perpendicular to the line of sight
The processing steps can be summarized as follows:
- Compute the area on the plane perpendicular to the line of sight for each pixel of the DEM
- Project the areas into the reference system of the image.
- Sum all the area contributions within same image pixel.
- Normalize the GTC image with respect to the computed areas divided by the image pixel size (in meters).

Note: 
- The Gamma Flatteing works properly if the input DEM has a pixel size smaller than the size of the pixel where the areas are summed. The default is the input Sentinel-1 pixel size.
- The size of the pixel does not correspond necessarily with the Sentinel-1 pixel image size. 
 

Input parameters:
- `correct_radiometry`: if `True`, it activates the Gamma Flattening correction.
- `grouping_area_factor`: scaling factor for the size of the image pixel where the areas are summed. By default, the `grouping_area_factor` is `(1, 1)`, which corresponds to Sentinel-1 input product pixel size. The `grouping_area_factor` shall be increased if the DEM resolution is lower than the Sentinel-1 input product resolution to avoid gaps and distortions the normalization factor. It can be also used to to speed up the computation or  the DEM resolution is lower than the Sentinel-1 input product resolution.  


In [ ]:
rtc_path = os.path.basename(product_folder) + ".10m.RTC.tif"

apps.backward_geocode_sentinel1(
    grd_local_path,
    measurement_group,
    dem_UTM_urlpath,
    interp_method="nearest",
    correct_radiometry="gamma",
    output_urlpath=rtc_path,
    grouping_area_factor=(3, 3),
)

In [ ]:
rtc = xr.open_dataarray(rtc_path, cache=False).drop("band")
rtc.plot(vmax=0.4)

### Comparison between GTC and RTC

In [ ]:
f, axes = plt.subplots(nrows=1, ncols=2, figsize=(30, 12))

_ = gtc.plot(ax=axes[0], vmax=0.4)
axes[0].grid(c="red")
plt.title("GTC")


_ = rtc.plot(ax=axes[1], vmax=0.4)
axes[1].grid(c="red")
plt.title("RTC")


plt.tight_layout()

### Comparison between sarsen RTC and Planetary Computer RTC


#### Open Planetary Sentinel-1 RTC

In [ ]:
rtc_account_name = "sentinel1euwestrtc"
rtc_storage_container = "sentinel1-grd-rtc"
rtc_token = planetary_computer.sas.get_token(
    rtc_account_name, rtc_storage_container
).token

rtc_product_folder = f"{rtc_storage_container}/{product_folder}"

rtc_fs = adlfs.AzureBlobFileSystem(rtc_account_name, credential=rtc_token)
rtc_fs.ls(rtc_product_folder)

In [ ]:
rtc_local_path = os.path.join(tmp_dir, rtc_product_folder)
rtc_fs.get(f"{rtc_product_folder}", rtc_local_path, recursive=True)

In [ ]:
rtc_pc = xr.open_dataarray(
    rtc_local_path + "/measurement/iw-vv.rtc.tiff", cache=False
).drop("band")
rtc_pc = rtc_pc.sel(dem_UTM_corners)
rtc_pc

#### Plot sarsen RTC and Planetary Computer RTC

In [ ]:
f, axes = plt.subplots(nrows=1, ncols=2, figsize=(30, 12))

_ = rtc_pc.plot(ax=axes[0], vmax=0.4)
axes[0].grid(c="red")
plt.title("Planetary Computer RTC")

_ = rtc.plot(ax=axes[1], vmax=0.4)
axes[1].grid(c="red")
plt.title("sarsen RTC")

plt.tight_layout()